The goal of this worksheet is to construct a smooth quartic birational to a curve of degree $8$ with six $\mathbb{E}_6$ points invariant by an automorphism of order $3$. We recover previous computations. The curve with equation $F_0(x,y,z)=0$ is invariant by $(x,y,z)\mapsto(\zeta x, \zeta^{-1} y, z)$, where $\zeta^3=1$.

Some long equations are skipped if the variable ``verif`` is set to ``True``. It can be changed at any time.

In [1]:
verif = False
f, F0, F2a = load('files3/octica-3-final.sobj')

In [2]:
K0 = F0.base_ring()
K0.inject_variables()
R = F0.parent()
R.inject_variables()

Defining b0
Defining x, y, z


The field $K_0$ contains the cubic roots of unity.

In [3]:
Cy3 = CyclotomicField(3, 'bb')
zeta3 = Cy3.embeddings(K0)[0](Cy3.gen())

We need to place the singular points. Let us look for the $x$-coordinates ($y=1$).

In [4]:
%%time 
T.<t> = K0[]
dis0 = F0.discriminant(z)(x=t, y=1)
dis01 = dis0.derivative(t, 7)
disgcd = dis01.gcd(dis0)
dis = [f[0] for f in disgcd.factor()]
len(dis)

CPU times: user 1.68 s, sys: 6.01 ms, total: 1.69 s
Wall time: 1.69 s


2

The coordinates of the singular points live in an extension. Let us construct it.

In [5]:
%%time
p = dis[0]
K1 = K0.extension(p, 's6')
xs0 = [K1.gen(0) * a for a in (1, zeta3, zeta3^2)]
L1.<a6> = K1.absolute_field()
R1 = R.change_ring(L1)
R1.inject_variables(verbose=False)
T1 = T.change_ring(L1)
T1.inject_variables(verbose=False)
xs1 = [L1(a) for a in xs0]
zeta3a = L1(K1(zeta3))
q = dis[1].change_ring(K1).change_ring(L1)
K2 = L1.extension(q, 's7') 
xs2 = [K2(a) for a in xs1] + [K2.gen(0) * a for a in (1, zeta3a, zeta3a^2)]
L2.<s7> = K2.absolute_field()
F1 = F0.change_ring(K1).change_ring(L1).change_ring(K2).change_ring(L2)
xs = [L2(a) for a in xs2]
zeta3b = L2(K2(zeta3a))
R2 = R1.change_ring(L2)
R2.inject_variables(verbose=False)
T2 = T1.change_ring(L2)
T2.inject_variables(verbose=False)

CPU times: user 1min 7s, sys: 8.01 s, total: 1min 15s
Wall time: 1min 16s


In [6]:
dis[0].change_ring(K1).change_ring(L1).change_ring(K2).change_ring(L2)(t=xs[0])

0

Once in the right field $L_2$ we construct the list $L_6$ containing the six singular points.

In [7]:
x0 = xs[0]
f = F1(x=x0, y=1)(z=t)
f1 = f.derivative(t)
f0 = f.gcd(f1)
z0 = f0.roots(multiplicities=False)[0]
sing = [z0^-1 * vector([a * x0, a^2, z0]) for a in (1, zeta3b, zeta3b^2)]
x0 = xs[3]
f = F1(x=x0, y=1)(z=t)
f1 = f.derivative(t)
f0 = f.gcd(f1)
z0 = f0.roots(multiplicities=False)[0]
sing += [z0^-1 * vector([a * x0, a^2, z0]) for a in (1, zeta3b, zeta3b^2)]

In [8]:
load('functions/functions1.sage')

We construct the six conics passing through five of the six singular points and we check their behaviour by the action.

In [9]:
conicas = [conica(R2, sing[: j] + sing[j + 1:]) for j in range(6)]
accion = conicas[1](x=zeta3b * x, y=zeta3b^-1 * y) == conicas[0]
accion = accion and conicas[2](x=zeta3b^-1 * x, y=zeta3b * y) == conicas[0]
accion = accion and conicas[4](x=zeta3b * x, y=zeta3b^-1 * y) == conicas[3]
accion = accion and conicas[5](x=zeta3b^-1 * x, y=zeta3b * y) == conicas[3]
accion

True

We construct a basis for the equations of the quintics with double points at the singular points. Then we get another basis passing through the vertices of the coordinate system.

In [10]:
rct0 = reducir(sing[1].cross_product(sing[2])) * vector([x, y, z])
rct1 = reducir(sing[0].cross_product(sing[2])) * vector([x, y, z])
rct2 = reducir(sing[0].cross_product(sing[1])) * vector([x, y, z])
p0 = rct0 * conicas[1] * conicas[2]
p1 = conicas[0] * rct1 * conicas[2]
p2 = conicas[0] * conicas[1] * rct2
A = Matrix([[p(*vv) for vv in ((1, 0, 0), (0, 1, 0), (0, 0, 1))] for p in (p0, p1, p2)])
qq = [col * vector([p0, p1, p2]) for col in reversed(A.columns())]
for q0 in qq:
    print([q0(*vv) == 0 for vv in ((1, 0, 0), (0, 1, 0), (0, 0, 1))])

[False, True, True]
[True, False, True]
[True, True, False]


We normalize the equations to have the value $1$ at $[1:1:1]$.

In [11]:
qq = [R2(q0 / q0(1, 1, 1)) for q0 in qq]

We check the behaviour with the action.

In [12]:
F1 - F1(x=zeta3b * x, y=y * zeta3b^-1)

0

In [13]:
[zeta3b^j * q0 - q0(x=zeta3b * x, y=y * zeta3b^-1) for j, q0 in zip((-1, 1, 0), qq)]

[0, 0, 0]

Let us define the birational map for the blowing-up of the six singular points and the blowing down of the conics, for pull-back equations, ``birat1``, and images of points, ``birat_punto1``

In [14]:
def birat1(g):
    return g.subs({vr: q0 for vr, q0 in zip(R2.gens(), qq)})
def birat_punto1(vv):
    return vector(q0(*vv) for q0 in qq)

In [15]:
V = L2^3
[reducir(birat_punto1(vv)) == vv for vv in V.gens()]

[True, True, True]

Let us cehck the degrees of the minimal polynomials of the coordinates of the birational transformation. The coefficients are in $K_0$!

In [16]:
for q0 in qq:
    print([m.minpoly().degree() for m in q0.coefficients()])

[8, 8, 8, 8, 8, 8, 8]
[8, 8, 8, 8, 8, 8, 8]
[8, 8, 4, 8, 8, 4, 4]


In [17]:
h = K0.embeddings(L2)[1]

In [18]:
cfs1 = vector(F1.coefficients())
cfs0 = cfs1.apply_map(lambda _: h.preimage(_))
F0 == cfs0 * vector(F0.monomials())

True

We reconstruct the same map but with coefficients actually in $K_0$ which is much smaller.

In [19]:
qqa = []
for q0 in qq:
    cfs1 = vector(q0.coefficients())
    cfs0 = cfs1.apply_map(lambda _: h.preimage(_))
    q1 = cfs0 * vector([R(m) for m in q0.monomials()])
    qqa.append(q1)

In [20]:
def birat1a(g):
    return g.subs({vr: q0 for vr, q0 in zip(R.gens(), qqa)})
def birat_punto1a(vv):
    return vector(q0(*vv) for q0 in qqa)

We consider the image of a general point in each conic, in order to compute the inverse Cremona transformation. We choose distinct general points and we check that the result does not change.

In [21]:
sing1 = []
for j, con in enumerate(conicas):
    k = (j + 1) % 6
    x2 = L2(x.reduce([R2(con(y=sing[k][1] / sing[k][0] * x, z=1) / (sing[k][0] - sing[k][2] * x))]))
    y2 = sing[k][1] / sing[k][0] * x2
    sing1.append(reducir3(birat_punto1([x2, y2, 1])))

In [22]:
sing1a = []
for j, con in enumerate(conicas):
    k = (j + 2) % 6
    x2 = L2(x.reduce([R2(con(y=sing[k][1] / sing[k][0] * x, z=1) / (sing[k][0] - sing[k][2] * x))]))
    y2 = sing[k][1] / sing[k][0] * x2
    sing1a.append(reducir3(birat_punto1([x2, y2, 1])))

In [23]:
sing1 == sing1a

True

We construct the inverse of the Cremona transformation, since it is not involutive.

In [24]:
conicas1 = [conica(R2, sing1[: j] + sing1[j + 1:]) for j in range(6)]
accion = accion and conicas1[2](x=zeta3b * x, y=zeta3b^-1 * y) == conicas1[0]
accion = accion and conicas1[1](x=zeta3b^-1 * x, y=zeta3b * y) == conicas1[0]
accion = accion and conicas1[5](x=zeta3b * x, y=zeta3b^-1 * y) == conicas1[3]
accion = accion and conicas1[4](x=zeta3b^-1 * x, y=zeta3b * y) == conicas1[3]
accion

True

In [25]:
rct01 = reducir(sing1[1].cross_product(sing1[2])) * vector([x, y, z])
rct11 = reducir(sing1[0].cross_product(sing1[2])) * vector([x, y, z])
rct21 = reducir(sing1[0].cross_product(sing1[1])) * vector([x, y, z])
p01 = rct01 * conicas1[1] * conicas1[2]
p11 = conicas1[0] * rct11 * conicas1[2]
p21 = conicas1[0] * conicas1[1] * rct21
A1 = Matrix([[p(*vv) for vv in ((1, 0, 0), (0, 1, 0), (0, 0, 1))] for p in (p01, p11, p21)])
qq1 = [col * vector([p01, p11, p21]) for col in reversed(A1.columns())]
for q0 in qq1:
    print([q0(*vv) == 0 for vv in ((1, 0, 0), (0, 1, 0), (0, 0, 1))])

[False, True, True]
[True, False, True]
[True, True, False]


In [26]:
qq1 = [q0 / q0(1, 1, 1) for q0 in qq1]

In [27]:
[zeta3b^j * q0 - q0(x=zeta3b * x, y=y * zeta3b^-1) for j, q0 in zip((-1, 1, 0), qq1)]

[0, 0, 0]

In [28]:
def birat2(g):
    return g.subs({vr: q0 for vr, q0 in zip(R2.gens(), qq1)})
def birat_punto2(vv):
    return vector(q0(*vv) for q0 in qq1)

In [29]:
for q0 in qq1:
    print([m.minpoly().degree() for m in q0.coefficients()])

[8, 8, 8, 8, 8, 8, 8]
[8, 8, 8, 8, 8, 8, 8]
[8, 8, 4, 8, 8, 4, 4]


We reconstruct it again with coefficients in $K_0$. 

In [30]:
qq1a = []
for q0 in qq1:
    cfs1 = vector(q0.coefficients())
    cfs0 = cfs1.apply_map(lambda _: h.preimage(_))
    q1 = cfs0 * vector([R(m) for m in q0.monomials()])
    qq1a.append(q1)

In [31]:
def birat2a(g):
    return g.subs({vr: q0 for vr, q0 in zip(R.gens(), qq1a)})
def birat_punto2a(vv):
    return vector(q0(*vv) for q0 in qq1a)

In [32]:
[q0.change_ring(K1).change_ring(L1).change_ring(K2).change_ring(L2) == q1 for q0, q1 in zip(qqa, qq)]

[True, True, True]

In [33]:
[q0.change_ring(K1).change_ring(L1).change_ring(K2).change_ring(L2) == q1 for q0, q1 in zip(qq1a, qq1)]

[True, True, True]

Until it is necessary we work in $K_0$.

In [34]:
R.inject_variables(verbose=False)

In [35]:
reducir(birat_punto1a(birat_punto2a(vector([1, 1, 1]))))

(1, 1, 1)

In [36]:
reducir(birat_punto2a(birat_punto1a(vector([1, 1, 1]))))

(1, 1, 1)

We check that the transformations are inverse. This is the output if `verif = True` with the time.
```
verification for x : True
verification for y : True
verification for z : True
CPU times: user 1min 7s, sys: 71.3 ms, total: 1min 7s
Wall time: 1min 8s
```

In [37]:
prodcon = prod(conicas)
prodcon = vector(h.preimage(a) for a in prodcon.coefficients()) * vector(R(m) for m in prodcon.monomials())
prodcon1 = prod(conicas1)
prodcon1 = vector(h.preimage(a) for a in prodcon1.coefficients()) * vector(R(m) for m in prodcon1.monomials())
verif = False
if verif: 
    x1 = birat1a(birat2a(x))
    x2 = birat2a(birat1a(x))
    x1 = x1 / x1.monomial_coefficient(x^25)
    x2 = x2 / x2.monomial_coefficient(x^25)
    x3 = x * prodcon^2
    x3 = x3 / x3.monomial_coefficient(x^25)
    x4 = x * prodcon1^2
    x4 = x4 / x4.monomial_coefficient(x^25)
    print ('verification for', x, ':', x1 == x3 and x2 == x4)
    y1 = birat1a(birat2a(y))
    y2 = birat2a(birat1a(y))
    y1 = y1 / y1.monomial_coefficient(y^25)
    y2 = y2 / y2.monomial_coefficient(y^25)
    y3 = y * prodcon^2
    y3 = y3 / y3.monomial_coefficient(y^25)
    y4 = y * prodcon1^2
    y4 = y4 / y4.monomial_coefficient(y^25)
    print ('verification for', y, ':', y1 == y3 and y2 == y4)
    z1 = birat1a(birat2a(z))
    z2 = birat2a(birat1a(z))
    z1 = z1 / z1.monomial_coefficient(z^25)
    z2 = z2 / z2.monomial_coefficient(z^25)
    z3 = z * prodcon^2
    z3 = z3 / z3.monomial_coefficient(z^25)
    z4 = z * prodcon1^2
    z4 = z4 / z4.monomial_coefficient(z^25)
    print ('verification for', z, ':', z1 == z3 and z2 == z4)

We compute the transformation and get the equation of the quartic. Instead of factorizing we divide three times by the product of the conics. For the first computation:
```
CPU times: user 3min 32s, sys: 479 ms, total: 3min 32s
Wall time: 3min 32s
```
The second one:
```
CPU times: user 55.7 s, sys: 142 ms, total: 55.9 s
Wall time: 55.9 s
```
The third one:
```
CPU times: user 27.5 s, sys: 47.6 ms, total: 27.5 s
Wall time: 28.1 s
```
The fourth one:
```
CPU times: user 8.84 s, sys: 13.9 ms, total: 8.86 s
Wall time: 8.89 s
```

In [38]:
if verif:
    G0 = birat2a(F0)

In [39]:
if verif:
    G0a = R(G0 / prodcon1)

In [40]:
if verif:
    G0b = R(G0a / prodcon1)

In [41]:
if verif:
    G0c = R(G0b / prodcon1)
    save(G0c, 'files3/G0c')
else:
    G0c = load('files3/G0c.sobj')

We wanto to obtain a simpler equation.

In [42]:
G0c = G0c / G0c.monomial_coefficient(z^4)

The equation must satisfy symmetry conditions.

In [43]:
k0 = K0.automorphisms()[1]

In [44]:
def verifica(G):
    a22 = G.monomial_coefficient(x^2 * y^2)    
    a30 = G.monomial_coefficient(x^3 * z)
    a03 = G.monomial_coefficient(y^3 * z)
    a11 = G.monomial_coefficient(x * y * z^2)
    return a22 == k0(a22) and a11 == k0(a11) and a30 == k0(a03)

In [45]:
verifica(G0c)

True

In [46]:
G0c.monomials()

[x^2*y^2, x^3*z, y^3*z, x*y*z^2, z^4]

The equation is too complicated, so we try to simplify it and we keep track of the changes.

In [47]:
xc = 1
zc = 1

In [48]:
b = G0c.monomial_coefficient(x^2 * y^2)
bf = b.factor()
print([a[1] for a in bf])

[2, 2, 12, 6, -3, -3, -3, -3, -4, 2, 2, 2, 2, -4, -4, 2, 2]


In [49]:
b = G0c.monomial_coefficient(x^2 * y^2)
bf = b.factor()
U = [a[0] for a in bf if a[1] == 2]
x0 = 1
y0 = 1
while U:
    a = U[0]
    a1 = k0(a)
    x0 *= a
    y0 *= a1
    U = [b for b in U if b != a and b != a1]
G0d = G0c(x=x / x0, y=y / y0)
xc *= x0
verifica(G0d)

True

In [50]:
b = G0d.monomial_coefficient(x^2 * y^2)
bf = b.factor()
print([a[1] for a in bf])

[-2, -2, 12, 6, -3, -3, -3, -3, -4, -2, -2, -2, -2, -4, -4, -2, -2]


In [51]:
b = G0d.monomial_coefficient(x^2 * y^2)
bf = b.factor()
U = [a[0] for a in bf if a[1] == -2]
x0 = 1
y0 = 1
while U:
    a = U[0]
    a1 = k0(a)
    x0 *= a
    y0 *= a1
    U = [b for b in U if b != a and b != a1]
G0e = G0d(x=x / x0, y=y / y0)
xc *= x0
verifica(G0e)

True

In [52]:
b = G0e.monomial_coefficient(x^2 * y^2)
bf = b.factor()
print([a[1] for a in bf])

[-6, -6, 12, 6, -3, -3, -3, -3, -4, -6, -6, -6, -6, -4, -4, -6, -6]


In [53]:
b = G0e.monomial_coefficient(x^2 * y^2)
bf = b.factor()
U = [a[0] for a in bf if a[1] == -4]
x0 = prod(U)^-1
y0 = k0(x0)
G0f = G0e(x=x / x0, y=y / y0)
xc *= x0
verifica(G0f)

True

In [54]:
b = G0f.monomial_coefficient(x^3 * z)
bf = b.factor()
print([a[1] for a in bf])

[-6, -6, 9, 3, -3, -3, -1, -1, -6, -3, -6, -3, -6, -3]


In [55]:
b = G0f.monomial_coefficient(x^3 * z)
bf = b.factor()
U = [a[0] for a in bf if a[1] % 3 == 0]
x0 = 1
for a, c in bf:
    if c % 3 == 0:
        x0 *= a^(c / 3)
y0 = k0(x0)
G0g = G0f(x=x / x0, y=y / y0)
xc *= x0
verifica(G0g)

True

In [56]:
G0g

(-1984329/59831*b0^7 - 31395081/239324*b0^6 - 13442895/59831*b0^5 - 52878987/119662*b0^4 - 61766289/119662*b0^3 - 487932885/239324*b0^2 - 24539031/119662*b0 - 29000121/59831)*x^2*y^2 + (-284213/478648*b0^7 - 956995/478648*b0^6 - 47481/12596*b0^5 - 770967/119662*b0^4 - 4534151/478648*b0^3 - 14443601/478648*b0^2 - 838373/239324*b0 - 864023/119662)*x^3*z + (-200029/478648*b0^7 - 971751/478648*b0^6 - 745651/239324*b0^5 - 883463/119662*b0^4 - 162685/25192*b0^3 - 16001773/478648*b0^2 - 683801/239324*b0 - 959277/119662)*y^3*z + (11727/25192*b0^7 + 877815/478648*b0^6 + 753219/239324*b0^5 + 365121/59831*b0^4 + 3441855/478648*b0^3 + 13495401/478648*b0^2 + 681381/239324*b0 + 751077/119662)*x*y*z^2 + z^4

In [57]:
G1 = G0c(x=x / xc, y=y/ k0(xc))
G1 == G0g

True

In [58]:
[G0c.change_ring(K1).change_ring(L1).change_ring(K2).change_ring(L2)(*vv) for vv in sing1]

[0, 0, 0, 0, 0, 0]

In [59]:
xd = L2(K2(L1(K1(xc))))
yd = L2(K2(L1(K1(k0(xc)))))
sing1b = []
for vv in sing1:
    a, b, c = vv
    sing1b.append(vector([a * xd, b * yd, c]))

In [60]:
[G1.change_ring(K1).change_ring(L1).change_ring(K2).change_ring(L2)(*vv) for vv in sing1b]

[0, 0, 0, 0, 0, 0]

In [61]:
qq1b = [q0(x=x / xc, y=y/ k0(xc)) for q0 in qq1a]
qq1b = [R(q0 / q0(1, 1, 1)) for q0 in qq1b]

These are the new coordinates for the inverse of the Cremona transformation. Let us compute the new coordinates of
the former Cremona transformation.

In [62]:
def birat2b(g):
    return g.subs({vr: q0 for vr, q0 in zip(R.gens(), qq1b)})
def birat_punto2b(vv):
    return vector(q0(*vv) for q0 in qq1b)

In [63]:
qq1c = [q0.change_ring(K1).change_ring(L1).change_ring(K2).change_ring(L2) for q0 in qq1b]
def birat2c(g):
    return g.subs({vr: q0 for vr, q0 in zip(R2.gens(), qq1c)})
def birat_punto2c(vv):
    return vector(q0(*vv) for q0 in qq1c)

We compute the conics in the new variable and the images of a general point.

In [64]:
R2.inject_variables(verbose=False)
conicas2 = [conica(R2, sing1b[: j] + sing1b[j + 1:]) for j in range(6)]

In [65]:
sing2 = []
for j, con in enumerate(conicas2):
    k = (j + 1) % 6
    x2 = L2(x.reduce([R2(con(y=sing1b[k][1] / sing1b[k][0] * x, z=1) / (sing1b[k][0] - sing1b[k][2] * x))]))
    y2 = sing1b[k][1] / sing1b[k][0] * x2
    sing2.append(reducir3(birat_punto2c([x2, y2, 1])))

In [66]:
conicas3 = [conica(R2, sing2[: j] + sing2[j + 1:]) for j in range(6)]

In [67]:
rct02 = reducir(sing2[1].cross_product(sing2[2])) * vector([x, y, z])
rct12 = reducir(sing2[0].cross_product(sing2[2])) * vector([x, y, z])
rct22 = reducir(sing2[0].cross_product(sing2[1])) * vector([x, y, z])
p03 = rct02 * conicas3[1] * conicas3[2]
p13 = conicas3[0] * rct12 * conicas3[2]
p23 = conicas3[0] * conicas3[1] * rct22
A2 = Matrix([[p(*vv) for vv in ((1, 0, 0), (0, 1, 0), (0, 0, 1))] for p in (p03, p13, p23)])
qq2 = [col * vector([p03, p13, p23]) for col in reversed(A2.columns())]
for q0 in qq2:
    print([q0(*vv) == 0 for vv in ((1, 0, 0), (0, 1, 0), (0, 0, 1))])

[False, True, True]
[True, False, True]
[True, True, False]


In [68]:
qq2 = [q0 / q0(1, 1, 1) for q0 in qq2]

In [69]:
[zeta3b^j * q0 - q0(x=zeta3b * x, y=y * zeta3b^-1) for j, q0 in zip((-1, 1, 0), qq2)]

[0, 0, 0]

In [70]:
def birat1c(g):
    return g.subs({vr: q0 for vr, q0 in zip(R2.gens(), qq2)})
def birat_punto1c(vv):
    return vector(q0(*vv) for q0 in qq2)

In [71]:
for q0 in qq2:
    print([m.minpoly().degree() for m in q0.coefficients()])

[8, 8, 8, 8, 8, 8, 8]
[8, 8, 8, 8, 8, 8, 8]
[8, 8, 4, 8, 8, 4, 4]


We reconstruct it again with coefficients in $K_0$. 

In [72]:
qq2a = []
for q0 in qq2:
    cfs1 = vector(q0.coefficients())
    cfs0 = cfs1.apply_map(lambda _: h.preimage(_))
    q1 = cfs0 * vector([R(m) for m in q0.monomials()])
    qq2a.append(q1)

In [73]:
def birat1b(g):
    return g.subs({vr: q0 for vr, q0 in zip(R.gens(), qq2a)})
def birat_punto1b(vv):
    return vector(q0(*vv) for q0 in qq2a)

In [74]:
[q0.change_ring(K1).change_ring(L1).change_ring(K2).change_ring(L2) == q1 for q0, q1 in zip(qq2a, qq2)]

[True, True, True]

In [75]:
[q0.change_ring(K1).change_ring(L1).change_ring(K2).change_ring(L2) == q1 for q0, q1 in zip(qq1b, qq1c)]

[True, True, True]

Until it is necessary we work in $K_0$.

In [76]:
R.inject_variables(verbose=False)

In [77]:
reducir(birat_punto1b(birat_punto2b(vector([1, 1, 1]))))

(1, 1, 1)

In [78]:
reducir(birat_punto2b(birat_punto1b(vector([1, 1, 1]))))

(1, 1, 1)

We check that the transformations are inverse.

We can verify if they are inverse of each other

In [79]:
prodcona = prod(conicas3)
prodcona = vector(h.preimage(a) for a in prodcona.coefficients()) * vector(R(m) for m in prodcona.monomials())
prodcon1a = prod(conicas2)
prodcon1a = vector(h.preimage(a) for a in prodcon1a.coefficients()) * vector(R(m) for m in prodcon1a.monomials())
verif = False
if verif: 
    x1 = birat1b(birat2b(x))
    x2 = birat2b(birat1b(x))
    x1 = x1 / x1.monomial_coefficient(x^25)
    x2 = x2 / x2.monomial_coefficient(x^25)
    x3 = x * prodcona^2
    x3 = x3 / x3.monomial_coefficient(x^25)
    x4 = x * prodcon1a^2
    x4 = x4 / x4.monomial_coefficient(x^25)
    print ('verification for', x, ':', x1 == x3 and x2 == x4)
    y1 = birat1b(birat2b(y))
    y2 = birat2b(birat1b(y))
    y1 = y1 / y1.monomial_coefficient(y^25)
    y2 = y2 / y2.monomial_coefficient(y^25)
    y3 = y * prodcona^2
    y3 = y3 / y3.monomial_coefficient(y^25)
    y4 = y * prodcon1a^2
    y4 = y4 / y4.monomial_coefficient(y^25)
    print ('verification for', y, ':', y1 == y3 and y2 == y4)
    z1 = birat1b(birat2b(z))
    z2 = birat2b(birat1b(z))
    z1 = z1 / z1.monomial_coefficient(z^25)
    z2 = z2 / z2.monomial_coefficient(z^25)
    z3 = z * prodcona^2
    z3 = z3 / z3.monomial_coefficient(z^25)
    z4 = z * prodcon1a^2
    z4 = z4 / z4.monomial_coefficient(z^25)
    print ('verification for', z, ':', z1 == z3 and z2 == z4)

In [80]:
H1 = birat1b(G1)

In [81]:
H1a = R(H1/prodcona)

In [82]:
H1b = H1a / H1a.monomial_coefficient(z^8)

In [83]:
dis0 = H1b.discriminant(z)

In [84]:
save(G1,'files3/cuartica')

In [85]:
G2 = G1.change_ring(K1).change_ring(L1).change_ring(K2).change_ring(L2)

In [86]:
R2.inject_variables(verbose=False)

In [87]:
prueba = conicas2[0].resultant(G2, z)

In [88]:
prueba1 = prueba(x=t, y=1)
prueba2 = prueba1.derivative(t, 2)
prueba3 = prueba1.gcd(prueba2)
xu = prueba3.roots()[0][0]

In [89]:
prueba = conicas2[0].resultant(G2, x)

In [90]:
prueba1 = prueba(z=t, y=1)
prueba2 = prueba1.derivative(t, 2)
prueba3 = prueba1.gcd(prueba2)
zu = prueba3.roots()[0][0]

In [91]:
conicas2[0](xu / zu, zu^-1, 1)

0

In [92]:
triples = []
for con in conicas2:
    res = con.resultant(G2, z)
    res1 = res(x=t, y=1)
    res2 = res1.derivative(t, 2)
    res3 = res1.gcd(res2)
    xu = res3.roots()[0][0]
    res = con.resultant(G2, x)
    res1 = res(z=t, y=1)
    res2 = res1.derivative(t, 2)
    res3 = res1.gcd(res2)
    zu = res3.roots()[0][0]
    triples.append(vector([xu / zu, zu^-1, 1]))

These are the triple intersections of the conics with the quartics.